# Testing Snowplow Algorithm

In [1]:
import initialize
import dual_graphs
import pandas as pd
import construction
from shortest_paths import ShortestPaths

In [2]:
%pwd
%cd ..

c:\Users\Sneez\Desktop\Snowplowing\Snowplow-Routing-Middleton


c:\Users\Sneez\Desktop\Snowplowing\Snowplow-Routing-Middleton\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
instance = "smalltoy"

match instance:
    case "smallstreets":
        G = initialize.create_small_streets()
    case "fullstreets":
        G = initialize.create_full_streets()
    case "smalltoy":
        G = initialize.create_small_toy()
    case _:
        print("Invalid instance name")
        
approach = "genetic"

if approach == "genetic":
    G = initialize.add_multi_edges(G)
    if instance == "smallstreets" or instance == "fullstreets":
        G_DUAL = dual_graphs.create_dual_streets(G)
    else:
        G_DUAL = dual_graphs.create_dual_toy(G)

elif approach == "annealing":
    if instance == "smallstreets" or instance == "fullstreets":
        G_DUAL = dual_graphs.create_dual_streets(G, False, True)
    else:
        G_DUAL = dual_graphs.create_dual_toy(G, False, True)



In [4]:
print(G_DUAL)

MultiDiGraph with 15 nodes and 50 edges


In [5]:
sp = ShortestPaths(G_DUAL, False, True)

In [6]:
construction.process_node(G, 0, 1)

([(1,
   2,
   0,
   {'distance': 1,
    'priority': 1,
    'travel_time': 2,
    'passes_rem': 1,
    'salt_per': 1,
    'deadheading_passes': 0,
    'serviced': False}),
  (1,
   2,
   1,
   {'distance': 1,
    'priority': 3,
    'travel_time': 1,
    'passes_rem': 2,
    'salt_per': 1,
    'deadheading_passes': 0,
    'serviced': False}),
  (1,
   4,
   0,
   {'distance': 2,
    'priority': 2,
    'travel_time': 2,
    'passes_rem': 1,
    'salt_per': 1,
    'deadheading_passes': 0,
    'serviced': False})],
 3,
 4)

In [7]:
construction.route_generation(G, sp)

([[0 --> 2. (0) Deadhead: False. Had options: False. Salt value: 100,
   2 --> 1. (0) Deadhead: False. Had options: True. Salt value: 99,
   1 --> 2. (1) Deadhead: False. Had options: True. Salt value: 98,
   2 --> 1. (1) Deadhead: False. Had options: True. Salt value: 97,
   1 --> 2. (0) Deadhead: False. Had options: True. Salt value: 96,
   2 --> 3. (1) Deadhead: False. Had options: True. Salt value: 95,
   3 --> 4. (0) Deadhead: False. Had options: True. Salt value: 94,
   4 --> 1. (0) Deadhead: False. Had options: True. Salt value: 93,
   1 --> 4. (0) Deadhead: False. Had options: True. Salt value: 92,
   4 --> 3. (0) Deadhead: False. Had options: True. Salt value: 91,
   3 --> 2. (0) Deadhead: False. Had options: True. Salt value: 90,
   2 --> 3. (0) Deadhead: False. Had options: True. Salt value: 89,
   3 --> 2. (1) Deadhead: False. Had options: True. Salt value: 88,
   2 --> 0. (0) Deadhead: False. Had options: True. Salt value: 87]],
 [[0 --> 2. (0) Deadhead: False. Had options